In [1]:
# IMPORTS
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
# DATASET
df = pd.read_csv("../datasets/stroke/healthcare-dataset-stroke-data_train.csv")
df.drop("id", axis=1, inplace=True)

### BMI por edad (medianas)
* Hasta:  10 años - BMI: 18.30  
* Hasta:  20 años - BMI: 23.80  
* Hasta:  30 años - BMI: 26.70  
* Hasta:  40 años - BMI: 29.70  
* Hasta:  50 años - BMI: 30.00  
* Hasta:  60 años - BMI: 30.90  
* Hasta:  70 años - BMI: 30.00  
* Mayor 70 años - BMI: 28.30  
* Mayor 30 años - BMI: 29.80  

Hago grupos hasta 10, hasta 20, hasta 30 y mayor a 30 para rellenar NaNs.

In [14]:
bmi_fill = {
    "hasta_10": 18.3,
    "hasta_20": 23.8,
    "hasta_30": 26.7,
    "mayor_30": 29.8,
}

In [29]:
# LLENO NaNs SEGUN EDAD
df.bmi.fillna(0, inplace=True)
for i in range(len(df)):
    if df.bmi.iloc[i] == 0:
        if (df.age.iloc[i]<=10):
            df.bmi.iloc[i] = bmi_fill["hasta_10"]
        elif (df.age.loc[i]<=20)&(df.age.loc[i]>10):
            df.bmi.iloc[i] = bmi_fill["hasta_20"]
        elif (df.age.loc[i]<=30)&(df.age.loc[i]>20):
            df.bmi.iloc[i] = bmi_fill["hasta_30"]
        else:
            df.bmi.iloc[i] = bmi_fill["mayor_30"]

c:\users\suare\myscripts\ml\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## ENCODING

In [36]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [42]:
ohe = OneHotEncoder()
work_type_ohe = ohe.fit_transform(df[["work_type"]])

<4088x5 sparse matrix of type '<class 'numpy.float64'>'
	with 4088 stored elements in Compressed Sparse Row format>